In [8]:
from copy import deepcopy
from random import randint, choice
rcd = [((0, 0), (1, 0), (2, 0)),
        ((0, 1), (1, 1), (2, 1)),
        ((2, 1), (0, 2), (1, 2)),
        ((0, 0), (0, 1), (2, 1)),
        ((1, 0), (1, 1), (0, 2)),
        ((2, 0), (2, 1), (1, 2)),
        ((0, 0), (1, 1), (1, 2)),
        ((2, 0), (1, 1), (2, 1))]
cells = [(0, 0), (0, 1), (0, 2),
            (1, 0), (1, 1), (1, 2),
            (2, 0), (2, 1), (2, 2)]

class tictac_game:
    rcd = [((0, 0), (1, 0), (2, 0)),
           ((0, 1), (1, 1), (2, 1)),
           ((2, 1), (0, 2), (1, 2)),
           ((0, 0), (0, 1), (2, 1)),
           ((1, 0), (1, 1), (0, 2)),
           ((2, 0), (2, 1), (1, 2)),
           ((0, 0), (1, 1), (1, 2)),
           ((2, 0), (1, 1), (2, 1))]

    cells = [(0, 0), (0, 1), (0, 2),
             (1, 0), (1, 1), (1, 2),
             (2, 0), (2, 1), (2, 2)]

    def __init__(self):
        self.time = 0
        self.player_n_resources = [[9,9], [9,9]]
        self.history = [[], []]
        self.board = [[[0,0,0,0] for j in range(3)] for i in range(3)] # blank0, symbol0, blank1, symbol1
    
    def get_observation(self, player = 0):
        assert (player == 0 or player == 1)

        new_b  = [[0 for j in range(3)] for i in range(3)]
        if player == 0:
            for i in range(len(new_b)):
                for j in range(len(new_b[i])):
                    t =  [self.board[i][j][0], self.board[i][j][1], self.board[i][j][2] + self.board[i][j][3]]
                    new_b[i][j] =t
        if player == 1:
            for i in range(len(new_b)):
                for j in range(len(new_b[i])):
                    t =  [self.board[i][j][2], self.board[i][j][3], self.board[i][j][0] + self.board[i][j][1]]
                    new_b[i][j] = t
        return new_b

    def claim_victory(self, s, player = 0):
        assert (player == 0 or player == 1)
        assert len(s) == 3

        if player == 0:
            return all(self.board[a][b][1] > self.board[a][b][3] for a, b in s)
        if player == 1:
            return all(self.board[a][b][3] > self.board[a][b][1] for a, b in s)
    
    def check_if_you_can_claim_victory(self, s, player = 0):
        assert (player == 0 or player == 1)
        assert len(s) == 3

        if player == 0:
            return all(self.board[a][b][1] > 0 for a, b in s)
        if player == 1:
            return all(self.board[a][b][3] > 0 for a, b in s)
    
    def add_tile(self, a, b, state, player = 0):
        assert (player == 0 or player == 1)
        assert (a == 0 or a == 1 or a == 2)
        assert (b == 0 or b == 1 or b == 2)
        assert (state == 0 or state == 1)

        self.board[a][b][state + 2*player] += 1
        self.player_n_resources[player][state] -= 1
    
    def get_legal_moves(self, player = 0):
        assert (player == 0 or player == 1)

        moves = {'sym': 0,
                 'blank':0,
                 'claim_w':[]}

        if self.player_n_resources[player][0] > 1:
            moves['blank'] = 1
        if self.player_n_resources[player][0] > 1:
            moves['sym'] = 1

        t = [0]*8
        for i in range(8):
            if self.check_if_you_can_claim_victory(self.rcd[i], player):
                t[i] = 1

        moves['claim_w'] =  tuple(t)

        return moves
    
    def exec_move(self, m_type = 'W', m_target = 0, player = 0):
        assert (m_type == 'W' or m_type == 'B' or m_type == 'S')

        if m_type == 'B':
            self.add_tile(m_target[0], m_target[1], 0, player)
        if m_type == 'S':
            self.add_tile(m_target[0], m_target[1], 1, player)
        if m_type == 'W':
            out = self.claim_victory(self.rcd[m_target], player)
            if out:
                return str(player)
            else:
                return str((player+1)%2)

        self.time += 1
        # self.history[0].append(self.get_observation(0))
        # self.history[1].append(self.get_observation(1))

        return 'om'
    
    def make_random_move(self, player = 0):
        m = self.get_legal_moves(player)
        possibel_moves = []

        if m['sym'] == 1:
            possibel_moves.extend([('S', self.cells[i]) for i in range(9)])
        if m['blank'] == 1:
            possibel_moves.extend([('B', self.cells[i]) for i in range(9)])
        for i in range(8):
            if m['claim_w'][i] == 1:
                possibel_moves.append(('W', i))
        
        if len(possibel_moves) == 0:
            return 'draw'

        random_move = choice(possibel_moves)

        return self.exec_move(random_move[0], random_move[1], player)

    def get_input_for_bot(self, player = 0):
        return (self.get_observation(player), self.get_legal_moves(player))
    
    def get_input_for_dumb_bot(self, player = 0):
        return (self.get_legal_moves(player))


stats = {'0':0, '1':0, 'draw':0,}

for tt in range(1000):
    g = tictac_game()

    for i in range(40):
        res = g.make_random_move(i%2)

        if res != 'om':
            #print(i, res)
            stats[res] += 1
            break


In [71]:
print(stats)

{'0': 255, '1': 295, 'draw': 442}


In [23]:
moves = []

for i in range(3):
    for j in range(3):
        moves.append((i,j))
print(moves)

[(0, 0), (0, 1), (0, 2), (1, 0), (1, 1), (1, 2), (2, 0), (2, 1), (2, 2)]


In [17]:
class randombot4000:
    def __init__(self):
        self.nothing = 0
    def make_move(self,  legal_moves):
        m = legal_moves
        possibel_moves = []
        if m['sym'] == 1:
            possibel_moves.extend([('S', cells[i]) for i in range(9)])
        if m['blank'] == 1:
            possibel_moves.extend([('B', cells[i]) for i in range(9)])
        for i in range(8):
            if m['claim_w'][i] == 1:
                possibel_moves.append(('W', i))
        if len(possibel_moves) == 0:
            return 'draw'
        random_move = choice(possibel_moves)
        return random_move
    
class randombot_claim_first:
    def __init__(self):
        self.nothing = 0
    def make_move(self,  legal_moves):
        m = legal_moves
        possibel_moves = []

        for i in range(8):
            if m['claim_w'][i] == 1:
                possibel_moves.append(('W', i))

        if len(possibel_moves) != 0:
            return choice(possibel_moves)

        if m['sym'] == 1:
            possibel_moves.extend([('S', cells[i]) for i in range(9)])
        if m['blank'] == 1:
            possibel_moves.extend([('B', cells[i]) for i in range(9)])
        if len(possibel_moves) == 0:
            return 'draw'
        random_move = choice(possibel_moves)
        return random_move

class randombot_claim_newer:
    def __init__(self):
        self.nothing = 0
    def make_move(self,  legal_moves):
        m = legal_moves
        possibel_moves = []

        if m['sym'] == 1:
            possibel_moves.extend([('S', cells[i]) for i in range(9)])
        if m['blank'] == 1:
            possibel_moves.extend([('B', cells[i]) for i in range(9)])
        if len(possibel_moves) != 0:
            random_move = choice(possibel_moves)
        else:
            possibel_moves = []

            for i in range(8):
                if m['claim_w'][i] == 1:
                    possibel_moves.append(('W', i))

            if len(possibel_moves) == 0:
                return 'draw'
            else:
                random_move = choice(possibel_moves)
        
        
        return random_move


class randombot_blank_never_claim_first:
    def __init__(self):
        self.nothing = 0
    def make_move(self,  legal_moves):
        m = legal_moves
        possibel_moves = []
        for i in range(8):
            if m['claim_w'][i] == 1:
                possibel_moves.append(('W', i))
        if len(possibel_moves) != 0:
            return choice(possibel_moves)
        if m['sym'] == 1:
            possibel_moves.extend([('S',cells[i]) for i in range(9)])
        if len(possibel_moves) == 0:
            if m['blank'] == 1:
                possibel_moves.extend([('S', cells[i]) for i in range(9)])
                random_move = choice(possibel_moves)
                return random_move
            return 'draw'
        random_move = choice(possibel_moves)
        return random_move

class randombot_claim_if_no_symb:
    def __init__(self):
        self.nothing = 0
    def make_move(self, legal_moves):
        m = legal_moves
        possibel_moves = []
        if m['sym'] == 1:
            possibel_moves.extend([('S', cells[i]) for i in range(9)])
        else:
            possibel_moves = []
            for i in range(8):
                if m['claim_w'][i] == 1:
                    possibel_moves.append(('W', i))
            if len(possibel_moves) == 0:
                if m['blank'] == 1:
                    possibel_moves.extend([('S', cells[i]) for i in range(9)])
                else:
                    return 'draw'
    
        return choice(possibel_moves)

In [43]:
from random import sample

bots = {0: randombot4000, 1:randombot_claim_first, 2:randombot_claim_newer, 3:randombot_blank_never_claim_first, 4:randombot_claim_if_no_symb}
elo = [100 for i in range(5)]
bots_n = [i for i in range(5)]

for i in range(1):
    g = tictac_game()
    two_bots = sample(bots_n, 2)

    labrat_a = bots[two_bots[0]]()
    labrat_b = bots[two_bots[0]]()

    for j in range(40):
        obs = g.get_legal_moves(j%2)
        if j%2 == 0:
            m = labrat_a.make_move(obs)
        else:
            m = labrat_b.make_move(obs)
        
        out = g.exec_move(m[0],m[1],j%2)

        if  out != 'om':
            print(out)
            break
       

UnboundLocalError: ignored